In [ ]:
import neuroglancer
import neuroglancer.static_file_server
import numpy as np
import SimpleITK as sitk
import xml.etree.ElementTree as ET
from pathlib import Path
from pytools import HedwigZarrImages

from dask.distributed import LocalCluster
import jinja2

cluster = LocalCluster()
client = cluster.get_client()

In [ ]:
file_dir = Path("/Users/blowekamp/scratch/hedwig/TestData/Nanostringfiles/ROI Alignment Images for Brad/")
server = neuroglancer.static_file_server.StaticFileServer(
        static_dir=file_dir, bind_address="localhost", daemon=True
    )
viewer = neuroglancer.Viewer()

shader_parameter_cache = {}

In [ ]:

def homogeneous_identity(N) -> np.array:    
    return np.identity(N+1)[:N,:]


def sitk_offset_from_transform( tx: sitk.Transform ) :        
    return tx.TransformPoint((0,)*tx.GetDimension())

def sitk_transform_to_ng_transform( tx: sitk.Transform ):
    
  tx = tx.GetInverse()
  tx_matrix = np.array(tx.GetMatrix()).reshape(tx.GetDimension(),tx.GetDimension())

  M = homogeneous_identity(5)
  M[4,4] = tx_matrix[0,0]
  M[3,4] = tx_matrix[1,0]
  M[4,3] = tx_matrix[0,1]
  M[3,3] = tx_matrix[1,1]    

  tx_translation = sitk_offset_from_transform(tx)
  M[4,5] = (tx_translation[0])*1e3
  M[3,5] = (tx_translation[1])*1e3

  output_dimensions = neuroglancer.CoordinateSpace( names=["t'", "c^", "z", "y", "x"], units =["", "", "nm","nm", "nm"], scales=[1,1,1,1,1])
  ng_transform = neuroglancer.CoordinateSpaceTransform(output_dimensions=output_dimensions,
                                                     matrix = M)
  return ng_transform

def add_zarr_image(viewer_txn, zarr_filename, transform_filename=None ):
  
  zarr_root = Path(zarr_filename).parent
  zarr_key = int(Path(zarr_filename).name)
  
  layername = f"{zarr_root.name}/{zarr_key}"
  output_dimensions = neuroglancer.CoordinateSpace( names=["t'", "c^", "z", "y", "x"], units =["", "", "nm","nm", "nm"], scales=[1,1,1,1,1])
  
  if transform_filename:
    tx = sitk.ReadTransform(transform_filename)
    ng_transform = sitk_transform_to_ng_transform(tx)
  else:
    M = homogeneous_identity(5)
    ng_transform = neuroglancer.CoordinateSpaceTransform(output_dimensions=output_dimensions,
                                                     matrix = M)
    
  viewer_txn.layers[layername] = neuroglancer.ImageLayer(
            source=neuroglancer.LayerDataSource(f"zarr://{server.url}/{zarr_filename}",
                                               transform=ng_transform),
            shader=generate_ng_shader(file_dir/zarr_root, zarr_key),
        )
  

def add_roi_annotations(viewer_txn, ome_xml_filename, *, layername="roi annotation", reference_zarr=None):

    scales= [398, 396]
    if reference_zarr:
        
        zarr_root = Path(reference_zarr).parent
        zarr_key = int(Path(reference_zarr).name)
        hwz_images = HedwigZarrImages(zarr_root)
        hwz_image = hwz_images[list(hwz_images.get_series_keys())[zarr_key]]
        spacing_tczyx = hwz_image.spacing
        # select X and Y scales and convert to nm from um
        scales = [s*1e3 for s in spacing_tczyx[:2:-1]]
        

    xml_path = Path(ome_xml_filename)
    
    ns = {"OME": "http://www.openmicroscopy.org/Schemas/OME/2016-06"}
    with open(xml_path, "r") as fp:
        data = fp.read()
        xml_root = ET.fromstring(data)

    layer = neuroglancer.LocalAnnotationLayer(
                dimensions=neuroglancer.CoordinateSpace(
                    names=["x", "y"],
                    units="nm",
                    scales=scales,
                ),
                
        )
    
    viewer_txn.layers[layername]=layer
    


    # Coordinates are in the space of the original input image. The dimensions/CooridinateSpace map the index space to physical space.
    for roi in xml_root.iterfind("OME:ROI", ns):
        for r in roi.iterfind("./OME:Union/OME:Rectangle", ns):
            height = float(r.attrib["Height"])
            width = float(r.attrib["Width"])
            x = float(r.attrib["X"])
            y = float(r.attrib["Y"])
            
            a = (x, y)
            b = (x+width, y+height)
        for l in roi.iterfind("./OME:Union/OME:Label", ns):
            text = l.attrib["Text"]
            
        print(a,b)
        layer.annotations.append(
            neuroglancer.AxisAlignedBoundingBoxAnnotation(
                description=text,
                id=neuroglancer.random_token.make_random_token(),
                point_a=a,
                point_b=b
            )
        )    

In [ ]:

def generate_ng_shader(path, key):
    global  shader_parameter_cache
    
    hwz_images = HedwigZarrImages(path)
    hwz_image = hwz_images[list(hwz_images.get_series_keys())[key]]
    
    rgb_shader_code="""
void main() {
  emitRGB(vec3(toNormalized(getDataValue(0)),
               toNormalized(getDataValue(1)),
               toNormalized(getDataValue(2))));
}
"""
    
    if hwz_image.shader_type == "RGB":
        return rgb_shader_code    
        
    
    if (path, key) not in shader_parameter_cache:

        shader_parameter_cache[(path, key)] = hwz_image.neuroglancer_shader_parameters(middle_quantile=[0.01, 0.999])
    
    params = shader_parameter_cache[(path, key)]
    
    template = """
#uicontrol float brightness slider(default={{brightness}}, min=-1, max=1, step=0.1)
#uicontrol float contrast slider(default={{contrast}}, min=-3, max=3, step=0.1)
    
{% for channel in channelArray %}
#uicontrol bool {{channel.name}} checkbox(default=true)
#uicontrol vec3 color{{channel.channel}} color(default="{{channel.color}}")
#uicontrol invlerp invlerp{{channel.channel}}(range=[{{channel.range[0]}}, {{channel.range[1]}}], window=[{{channel.window[0]}}, {{channel.window[1]}}], channel={{channel.channel}}, clamp=true)
{% endfor %}

void main() {
    vec3 cum = vec3(0., 0., 0.);
    {% for channel in channelArray %}
    if ({{channel.name}})
    {
        cum += color{{channel.channel}} * invlerp{{channel.channel}}(getDataValue({{channel.channel}}));
    }
    {% endfor %}
    emitRGB((cum+brightness)*exp(contrast));
}
"""
    
    j2_template = jinja2.Template(template)
    shader_code = j2_template.render(params)
    
    return shader_code
  
    

In [ ]:

def add_sitk_image(img:sitk.Image, name="image", transform=None, shader_code=None):
    
    img = img.ToScalarImage(inPlace=False)
    assert(img.GetDimension()==3)

    dimensions=neuroglancer.CoordinateSpace(
                                           names=['y', 'x', 'c^'],
                                           units=["nm", "nm", ""],
                                           scales=img.GetSpacing()[::-1])
                            

    ng_transform = None
    M = homogeneous_identity(3)
    if transform:
        print(transform.GetName())
        if transform.GetName() == "TranslationTransform":
            transform = sitk.AffineTransform(np.identity(2, dtype=float).flatten(), transform.GetOffset())
         
        tx_matrix = np.array(transform.GetMatrix()).reshape(transform.GetDimension(),transform.GetDimension())

        M[1,1] = tx_matrix[0,0]
        M[0,1] = tx_matrix[1,0]
        M[1,0] = tx_matrix[0,1]
        M[0,0] = tx_matrix[1,1]    
        
        tx_translation = sitk_offset_from_transform(transform)
        tx_translation = [t/s for t,s in zip(tx_translation, img.GetSpacing()[1:])]
        #M[0,3] = tx_translation[1]
        #M[1,3] = tx_translation[0]
    
        print(tx_translation)
        
    ng_transform = neuroglancer.CoordinateSpaceTransform(output_dimensions=dimensions,
                                                         matrix = M)

    volume = neuroglancer.LocalVolume( sitk.GetArrayViewFromImage(img), dimensions=dimensions )
             
                                                                                     
    with viewer.txn() as s:
      layer = neuroglancer.ImageLayer(
            source=neuroglancer.LayerDataSource(volume, transform=ng_transform),
            shader=shader_code,
      )
      s.layers[name] = layer

In [ ]:
shader_parameter_cache = {}

In [ ]:
with viewer.txn() as s:
  dimensions = neuroglancer.CoordinateSpace( names=["x", "y", "z"], units =["nm","nm", "nm"], scales=[1,1,1])
  s.dimensions = dimensions
  s.layout = neuroglancer.DataPanelLayout("xy")
  
  s.layers.clear()

  add_zarr_image(s,"IA_P2_S1.ome.zarr/0")
  
  add_zarr_image(s, "IA_P2_S4.zarr/0", file_dir/"IA_P2_S4_0_to_roi.txt")
  add_zarr_image(s, "IA_P2_S4.zarr/1", file_dir/"IA_P2_S4_1_to_roi.txt")
  
  add_roi_annotations(s, Path(file_dir)/ "IA_P2_S1.ome.zarr/OME/METADATA.ome.xml",
                      layername="roi annotation",
                      reference_zarr=Path(file_dir)/"IA_P2_S4.ome.zarr"/"0")
print(viewer)


In [ ]:
with viewer.txn() as s:
  dimensions = neuroglancer.CoordinateSpace( names=["x", "y", "z"], units =["nm","nm", "nm"], scales=[1,1,1])
  s.dimensions = dimensions
  s.layout = neuroglancer.DataPanelLayout("xy")

  s.layers.clear()

  add_zarr_image(s, file_dir/"IA_P2_S2.ome.zarr/0", server_url=server.url)
  
  add_zarr_image(s, file_dir/"IA_P2_S2.zarr/0", server.url, file_dir/"IA_P2_S2_0_to_roi.txt")
  add_zarr_image(s, file_dir/"IA_P2_S2.zarr/1", server.url, file_dir/"IA_P2_S2_1_to_roi.txt")
 
  add_roi_annotations(s, Path(file_dir)/ "IA_P2_S2.ome.zarr/OME/METADATA.ome.xml")
print(viewer)


In [ ]:
with viewer.txn() as s:
  dimensions = neuroglancer.CoordinateSpace( names=["x", "y", "z"], units =["nm","nm", "nm"], scales=[1,1,1])
  s.dimensions = dimensions
  s.layout = neuroglancer.DataPanelLayout("xy")
  
  s.layers.clear()
  
  add_zarr_image(s, "IA_P2_S3.ome.zarr/0")
  
  add_zarr_image(s, "IA_P2_S3.zarr/0", file_dir/"IA_P2_S3_0_to_roi.txt")
  add_zarr_image(s, "IA_P2_S3.zarr/1", file_dir/"IA_P2_S3_1_to_roi.txt")

  add_roi_annotations(s, Path(file_dir)/ "IA_P2_S3.ome.zarr/OME/METADATA.ome.xml")

print(viewer)

In [ ]:
with viewer.txn() as s:
  dimensions = neuroglancer.CoordinateSpace( names=["x", "y", "z"], units =["nm","nm", "nm"], scales=[1,1,1])
  s.dimensions = dimensions
  s.layout = neuroglancer.DataPanelLayout("xy")

  s.layers.clear()

  add_zarr_image(s, "IA_P2_S4.ome.zarr/0")
  
  add_zarr_image(s, "IA_P2_S1.zarr/0", file_dir/"IA_P2_S1_0_to_roi.txt")
  add_zarr_image(s, "IA_P2_S1.zarr/1", file_dir/"IA_P2_S1_1_to_roi.txt")

  add_roi_annotations(s, Path(file_dir)/ "IA_P2_S4.ome.zarr/OME/METADATA.ome.xml")
print(viewer)

